Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# CSE 204 Lab 13: Unsupervised Learning - Clustering

<img src="https://raw.githubusercontent.com/adimajo/CSE204-2021/master/data/logo.jpg" style="float: left; width: 15%" />

[CSE204-2021](https://moodle.polytechnique.fr/course/view.php?id=12838) Lab session #13

J.B. Scoggins, Jesse Read, Adrien Ehrhardt, Jérémie Decock

## Introduction

In this lab, you will implement two unsupervised learning algorithms to cluster data points based on similarity criteria: k-means, and spectral k-means. While libraries such as scikit-learn provide facilities that implement these algorithms, they are simple enough for you to implement with numpy alone. Before you begin, import the required packages.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from random import randrange

### Datasets

Throughout this lab, you will use 3 simple datasets to test your algorithms. Run the code below to visualize each dataset. As you can see, the first dataset consists of 4 normally-distributed clusters of points with equal variance. The second represents two clusters, one stretched vertically, and one horizontally. Finally, the last dataset represents 3 clusters distributed in rings. For convenience, the three datasets are placed in a list called `datasets`. In the rest of the lab, you will be asked to implement 2 clustering algorithms and run them on these datasets.

In [ ]:
# Create a data set
N = 120

data1 = np.random.normal((0, 0), (0.5, 0.5), size=(N, 2))
data1 = np.append(data1, np.random.normal((5,0), (0.5, 0.5), size=(N, 2)), axis=0)
data1 = np.append(data1, np.random.normal((0,5), (0.5, 0.5), size=(N, 2)), axis=0)
data1 = np.append(data1, np.random.normal((5,5), (0.5, 0.5), size=(N, 2)), axis=0)

data2 = np.random.normal((2, 5), (0.25, 1), size=(N, 2))
data2 = np.append(data2, np.random.normal((5, 5), (1, 0.25), size=(N, 2)), axis=0)

radii = np.random.normal(0, 0.5,size=(N, 1))
radii = np.append(radii, np.random.normal(4, 0.5,size=(2 * N, 1)), axis=0)
radii = np.append(radii, np.random.normal(8, 0.5,size=(3 * N, 1)), axis=0)
angles = np.random.uniform(size=(6 * N, 1)) * 2.0 * np.pi
data3 = np.hstack([radii * np.cos(angles), radii * np.sin(angles)])

datasets = [data1, data2, data3]

fig, axes = plt.subplots(1, len(datasets), figsize=(10, 3))
for i,data in enumerate(datasets):
    axes[i].scatter(data[:, 0], data[:, 1])

## Part 1: The k-Means Algorithm

k-means is one of the simplest unsupervised learning algorithms that solves the well-known clustering problem. The algorithm defines an iterative process, where the following two steps are repeated at each iteration:
1. take each instance belonging to the dataset and assign it to the nearest centroid, and
2. re-calculate the centroids of each of the k clusters. 
Thus, the k centroids change their location step by step until nothing changes anymore.

More formally, suppose that we are given a dataset $\mathbf{X} = \Big( \boldsymbol{x}_1, \dots, \boldsymbol{x}_n \Big)$, where each $\boldsymbol{x}_i \in \mathbb{R}^d$. The goal of the k-means algorithm is to group the data into $k$ cohesive clusters, where $k$ is an input parameter of the algorithm. **Your task is to implement this algorithm**. Algorithm 1 gives the pseudocode.

___
### Algorithm 1: k-means

**Input**: The dataset $\mathbf{X} = \Big( \boldsymbol{x}_1, \dots, \boldsymbol{x}_n \Big)$ (where each $\boldsymbol{x}_i \in \mathbb{R}^d$) and the hyper-parameter $k$ (the number of clusters).<br>
**Output**: Clusters assignments (the cluster assigned to each element of $\mathbf{X}$).

1. Initialize cluster centroids ${\boldsymbol\mu}_1, \boldsymbol{\mu}_2, \ldots, \boldsymbol{\mu}_k$ by choosing $k$ instances of $\mathbf{X}$ randomly;

**Repeat:**

2. Assign each instance $\boldsymbol{x}_i \in \mathbf{X}$ to the closest centroid, i.e., $c_i = \text{argmin}_z \|\boldsymbol{x}_i - \boldsymbol{\mu}_z\|_2$;

3. Re-compute the centroids $\boldsymbol{\mu}_1, \ldots, \boldsymbol{\mu}_k$ of each cluster based on $\boldsymbol{\mu}_z = \Big(\sum_{\boldsymbol{x} \in \mathbf{C}_z} \boldsymbol{x}\Big)/|\mathbf{C}_z|$, where $\mathbf{C}_z = \{\boldsymbol{x}_i | z_i = z\}$ the $z$-th cluster with $z=1, \ldots, k$ and $|\mathbf{C}_z|$ the size of the $z$-th cluster.

**until** Centroids do not change (convergence).
___

In the algorithm above, $k$ is a parameter of the algorithm and corresponds to the number of clusters we want to find; the cluster centroids $\boldsymbol{\mu}_z$ represent our current guesses for the positions of the centers of the clusters. To initialize the cluster centroids (in step 1 of the algorithm), we could choose $k$ training examples randomly (without replacement such that these are initially different), and set the cluster centroids to be equal to the values of these $k$ examples. Of course, other initialization methods are also possible, such as the [kmeans++ technique](https://en.wikipedia.org/wiki/K-means%2B%2B). To find the closest centroid, a distance (or similarity) function should be defined, and typically the Euclidean distance is used.

Based on its notion of similarity, the problem of $k$-means clustering can be reduced to the problem of finding appropriate centroids. This, in turn, can be expressed as the task of minimizing the following objective function:
$$
     E(k) = \sum_{z=1}^{k} \sum_{\boldsymbol{x}_i \in \mathbf{C}_z}\| \boldsymbol{x}_i - \boldsymbol{\mu}_z \|_2^2.
$$

**Implement the two following functions.**

In [ ]:
def euclidean_distance(X1: np.ndarray, X2: np.ndarray) -> np.ndarray:
    """
    Distance Function
    -----------------
    Computes the Euclidean distance between two arrays of points.

    Return
    ------
    A 2D n by m array where entry [i, j] is the distance 
    from the i-th point in X1 to the j-th point in X2.
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    # return ...

*Python hints*:
   * [`numpy.random.choice()`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html)
   * [`numpy.argmin()`](https://numpy.org/doc/stable/reference/generated/numpy.argmin.html)
   * [`numpy.min()`](https://numpy.org/doc/stable/reference/generated/numpy.amin.html)
   * [`numpy.mean()`](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)

In [ ]:
def k_means(X: np.ndarray, k: int):
    """
    k Means
    --------

    
    Parameters
    ----------
    
    X : an n-by-d matrix of inputs
    k : the number of clusters to find


    Algorithm:
    ----------

    1. Initialize (choose) the centroids
    2. Implement a `while` loop such that, while centroids have not changed since the last iteration:
        - compute the distances of all points to each centroid
        - label each point (associate it with) the nearest centroid
        - recompute the centroids (i.e., average of points belonging to each centroid)

    Return
    ------
    Tuple (z, iters, err)

    z : a 1D vector of labels of length n (e.g. z[i] = C_z means "x_i is belongs to cluster C_z")
    iters : the number of iterations carried out until convergence
    err: the error E(k)
    """
    # Initialize (choose) the centroids
    # Iterate until convergence
    ## Compute distances: you should probably use :code:`euclidian_distance`
    ## Label the points to their respective closest center
    ## Compute new centroids (as the barycentre - mean of all coordinates - of the points belonging to this cluster)
    # Calculate cost
    # YOUR CODE HERE
    raise NotImplementedError()
    return z, iters, err

To test your implementation, run the following code which will plot the 3 datasets, trying different values of $k$. It will display the number of iterations until convergence (along with $k$ in the title). **Comment in the following cell if we were able to recover the true data structure.**

In [ ]:
fig, axes = plt.subplots(3, len(datasets), figsize=(10, 10))
for i,k in enumerate([2, 3, 4]):
    for j, data in enumerate(datasets):
        labels, iters, err = k_means(data, k)
        axes[i,j].scatter(data[:, 0], data[:, 1], c=labels, cmap='rainbow')
        axes[i,j].set_title('$k=%d$, iter$=%d$' % (k, iters))

YOUR ANSWER HERE

To minimize the function $E(k)$, we wish to determine suitable centroids $(\boldsymbol{\mu}_z)_1^k$ such that, if the data is partitioned into corresponding clusters $(\mathbf{C}_z)_1^k$, distances between data points and their closest cluster centroid become as small as possible.

The convergence of the $k$-means algorithm is highly dependent on the initialization of the centroids. It may converge to a local minimum of the objective function $E(k)$. One way to overcome this problem is to execute the algorithm several times, with different initializations of the centroids. 

Another issue is how to determine the number of clusters ($k$) of the dataset. Intuitively, increasing $k$ without penalty will always reduce the amount of error in the resulting clustering, to the extreme case of zero error if each data point is in its own cluster (*i.e.*, when $k=n$). A method to determine $k$ is the *elbow rule*, similar to the one we used in the previous lab for determining the number of principal components to retain. The idea is to examine  and compare the error given above for a number of cluster solutions. In general, as the number of clusters increases, the *sum of squared errors* (SSE) should decrease because clusters are, by definition, smaller. A plot of the SSE against a series of sequential cluster levels (*i.e.*, different values) can be helpful here. That is, an appropriate cluster solution could be defined as the one where the reduction in SSE slows dramatically. This produces an "elbow" in the plot of SSE against the different values of $k$. 

Implement the elbow rule to find an appropriate value for $k$, as follows:

1. Run k-means clustering for values of $k=1, \ldots, 10$ (see `k_list`).
2. For each $k$, calculate the total intra-cluster error ($E(k)$, given above, and append it to `costs`).
3. Plot the curve of $E(k)$ vs $k$.
4. Try to identify the location of a bend (elbow) in the plot -- this is generally considered as an indicator of the appropriate number of clusters.

In [ ]:
def plot_error(dataset: np.ndarray):
    costs = []
    k_list = range(1, 11)
    # YOUR CODE HERE
    raise NotImplementedError()
    plt.xlabel('$k$');

In [ ]:
plot_error(datasets[0])

Based on this plot, which value of $k$ do you retain?

YOUR ANSWER HERE

## Part 2: Gaussian Mixture Model

Suppose you observe and measure a number of beetles, recording their length in centimeters -- given below in the array `x`. You are curious to investigate if you have recorded more than one species, since you believe you are studying two distinct species. You decide to use Gaussian Mixture Models on the data to elaborate on this hypothesis and gain further insight into the beetle population, in particular the distribution of their length.

**Task: Implement Gaussian Mixture Models**

___
### Algorithm 2: Expectation-Maximization

**Input**: The dataset $\mathbf{X} = \Big( {x}_1, \dots, {x}_n \Big)$ (where each ${x}_i \in \mathbb{R}$ - the generalization to $\mathbb{R}^d$ is a bit more tedious), the hyper-parameter $k$ (the number of clusters), and the number of iterations $n_{\text{iter}}$.<br>
**Output**: Clusters means $\mu = (\mu_z)_1^k$, standard deviations $\sigma = (\sigma_z)_1^k$, partial memberships $r_{i, j}$ and prior probabilities $\pi_j$ (see below).

**Initialize**:
* Randomly initialize a 2D array `r` where `r[i, z]` represents the membership of sample $i$ to cluster $z$, *i.e.* $p(z_{i, z} | x_i, \mu_z, \sigma_z)$. For this array to express a probability, each row must sum to 1.
* Randomly initialize a 2D array `m` where `m[:, z]` represents the mean of the gaussian of cluster $z$, *e.g.* to 0 or to the mean of each coordinate.
* Randomly initialize a 2D array `s` where `s[:, z]` represents the standard deviation of the gaussian of cluster $z$, *e.g.* to 1 or to the standard deviation of each coordinate.
* Randomly initialize a 1D array `w` where `w[z]` represents the prior probability of belonging to cluster $z$, *i.e.* $p(z)$, *e.g.* to $1 / k$. For this array to express a probability, it must sum to 1.

**Repeat for `n_iter` iterations:**

E. Compute `r[i, z]` as $\mathbb{E}[z_i | x_i]$ by computing $\tilde{r}_{i, z} = \mathcal{N} \left( x_i | \mu_z, \sigma_z \right) \pi_z$ and then normalize $r_{i, z} = \dfrac{\tilde{r}_{i, z}}{\sum_{z'} \tilde{r}_{i, z'}}$. The pdf of the Gaussian distribution is given in `g` below.

M. Update the parameters `m[:, z]`, `s[:, z]` and `w[z]` using:
* $\mu_z = \dfrac{\sum_{i=1}^n r_{i, z} x_i}{\sum_{i=1}^n r_{i, z}}$;
* $\sigma_z = \dfrac{\sum_{i=1}^n r_{i, z} (x_i - \mu_z)^2}{\sum_{i=1}^n r_{i, z}}$;
* $\pi_z = \dfrac{\sum_{i=1}^n r_{i, z}}{n}$.
___


*Note:* Plotting code is provided.

*Hint:* See the [lecture slides](https://moodle.polytechnique.fr/pluginfile.php/477776/mod_resource/content/3/Slides_13c.pdf) and [notes](https://moodle.polytechnique.fr/pluginfile.php/477768/mod_resource/content/4/Slides_13c.pdf) regarding pseudocode and outline. Note that you may obtain slightly different results each time you run the algorithm due to the random initialization.

In [ ]:
# beetle lengths
x = np.array([1.57, 1.16, 1.30, 0.26, 0.20, 0.43, 0.48, 0.34, 0.44, 0.61, 1.80, 1.40, 1.10, 1.25, 1.69], 
             dtype=float)

In [ ]:
print(x)

In [ ]:
def g(x: float, m: float = 0.0, s: float = 1.0) -> float:
    """
    Gaussian probability density function

    Parameters
    ----------
    
    x : a float to evaluate the pdf
    m : the mean
    s : the standard deviation

    Return
    ------
    exp^(-(x-m)² / (2s)²) / sqrt(2s\pi)
    """
    return np.exp(- (x - m) ** 2 / (2 * s)) / np.sqrt(2 * s * np.pi)

In [ ]:
def gmm(X: np.ndarray, k: int, n_iter: int):
    """
    Gaussian Mixture Model
    ----------------------

    
    Parameters
    ----------
    
    X : a vector of n samples
    k : the number of clusters to find
    n_iter : the number of iterations to perform


    Algorithm:
    ----------


    Return
    ------
    Tuple (z, iters, err)

    z : a 1D vector of labels of length n (e.g. z[i] = c_j means "x_i is belongs to cluster c_j")
    iters : the number of iterations carried out until convergence
    err: the error E(k)
    """
    # Initialize
    # Iterate for n_iter iterations
    ## E: expected membership | params
    ## M: maximize likelihood of this membership with params

    # YOUR CODE HERE
    raise NotImplementedError()
    return m, s, r, w

In [ ]:
m, s, r, w = gmm(x, 2, 100)

In [ ]:
fig = plt.figure()
# points to plot the gaussians obtained after EM
xx = np.linspace(0, 2, num=100)

# first gaussian
plt.plot(xx, g(xx, m[0], s[0]), 'r-', label="$g_1$")
# second gaussian
plt.plot(xx, g(xx, m[1], s[1]), 'b-', label="$g_2$")
# gaussian mixture - dashed
plt.plot(xx, w[0] * g(xx, m[0], s[0]) + w[1] * g(xx, m[1], s[1]), 'm:', label="mixture")

# labels for the two bee species: the most probable class Z
y = np.argmax(r, axis=1)

plt.legend()
plt.scatter(x, np.zeros(x.shape[0]), c=y, label="beetles")
plt.show()

### Bonus : implement a generalization to $\mathbb{R}^d$

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Part 3: Spectral Clustering

Spectral clustering techniques make use of the *spectrum* (eigenvalues) of the similarity matrix of the data to perform dimensionality reduction before clustering in fewer dimensions. The similarity matrix is provided as an input and consists in a quantitative assessment of the relative similarity of each pair of points in the dataset.

Given a set of data points $\boldsymbol{x}_1, \ldots ,\boldsymbol{x}_n, \forall \boldsymbol{x}_i \in \mathbb{R}^d$ and some notion of similarity $s_{ij}$ between all pairs of data points $\boldsymbol{x}_i$ and $\boldsymbol{x}_j$, the intuitive goal of clustering is to divide the data points into several groups such that points in the same group are similar and points in different groups are dissimilar to each other. If we do not have more information than similarities between data points, a nice way of representing the data is in form of the similarity graph $G = (V, E)$. Each vertex $v_i$ in this graph represents a data point $\boldsymbol{x}_i$. Two vertices are connected if the similarity $s_{ij}$ between the corresponding data points $\boldsymbol{x}_i$ and $\boldsymbol{x}_j$ is positive or larger than a certain threshold, and the edge is weighted by $s_{ij}$. The problem of clustering can now be reformulated using the similarity graph: we want to find a partition of the graph such that the edges between different groups have very low weights (which means that points in different clusters are dissimilar from each other) and the edges within a group have high weights (which means that points within the same cluster are similar to each other).

### Creating the similarity graph

There are several popular constructions to transform a given set $\boldsymbol{x}_1, \ldots , \boldsymbol{x}_n, \forall \boldsymbol{x}_i \in \mathbb{R}^d$ of data points with pairwise
similarities $w_{ij}$ or pairwise distances $d_{ij}$ into a graph. When constructing similarity graphs the goal is to model the local neighborhood relationships between the data points. We will use the approach of the $k$-Nearest Neighbors graph. Here the goal is to connect vertex $\boldsymbol{x}_i$ with vertex $\boldsymbol{x}_j$ if $\boldsymbol{x}_j$ is among the $k$-nearest neighbors of $\boldsymbol{x}_i$. This definition leads to a directed graph, as the neighborhood relationship is not symmetric. The most common way to deal with this, is to simply ignore the directions of the edges; that is, we connect $\boldsymbol{x}_i$ and $\boldsymbol{x}_j$ with an undirected edge if $\boldsymbol{x}_i$ is among the $k$-nearest neighbors of $\boldsymbol{x}_j$ or if $\boldsymbol{x}_j$ is among the $k$-nearest neighbors of $\boldsymbol{x}_i$ (with weights $w_{ij} = w_{ji} = 1$). The resulting graph is what is usually called the $k$-nearest neighbors graph.

### The algorithm

The pseudocode of the spectral clustering algorithm is given as follows. In spectral clustering, the data is projected into a lower-dimensional space (the spectral/eigenvector domain) where they are easily separable, say using $k$-means.

Given a dataset $\mathbf{X}=\{ \boldsymbol{x}_1, \ldots, \boldsymbol{x}_n\}$, where  each $\boldsymbol{x}_i \in \mathbb{R}^d$ and hyper-parameter $k$:

1. Construct the similarity graph $G$ as described above. Let
    * $\mathbf{W} = (w_{ij})_1^n$ be the adjacency matrix of this graph.
    * $\mathbf{D}$ be the diagonal degree matrix of graph $G$, ie $\mathbf{D}_{ii} = \sum_j {w}_{ij}$ and $\mathbf{D}_{ij} = 0$ for $i \neq j$.
2. Compute the Laplacian matrix $\mathbf{L} = \mathbf{D} - \mathbf{W}$.
3. Apply[ eigenvalue decomposition](https://en.wikipedia.org/wiki/Eigendecomposition_of_a_matrix) to the Laplacian matrix $\mathbf{L}$.
4. Select the eigenvectors which correspond to $k$ smallest eigenvalues, which we denote by $\mathbf{U}$.
5. Apply $k$-means to $\mathbf{U}$ (as if the rows were instances / samples), thus finding clusters $\mathbf{C}_1, \mathbf{C}_2, \ldots, \mathbf{C}_k$.

*Hint*: Use the `euclidian_distance` function and the `k_means` implementation you wrote in Task 1.

*Python hints:*

* to find the `k` nearest neighbors of each $\boldsymbol{x}_i \in \mathbf{X}$, you can use the function [`np.argsort`](https://numpy.org/doc/stable/reference/generated/numpy.argsort.html) with the array that stores `d(x_i, x_j)` for $\boldsymbol{x}_i, \boldsymbol{x}_j \in \mathbf{X}$;
* you can use [`eigenvalues, eigenvectors = np.linalg.eig(L)`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html) to obtain the eigenvalues and eigenvectors of a (square) matrix `L`.

In [ ]:
def spectral_k_means(X, k, k_nn):
    """
    Spectral Clustering
    -------------------

    Parameters
    ----------

    X : the data
    k : the number of clusters to find
    k_nn : the number of k nearest-neighbours to consider in the graph construction

    Returns
    -------

    Similar to the `k_means` method above
    """
    # YOUR CODE HERE
    raise NotImplementedError()

To test your implementation, run the following code which will plot the 3 datasets, trying different values of $k$. It will display the number of iterations (done by $k$-means) until convergence (along with $k$ in the title).

Comment in the following cell if we were able to recover the true data structure.

In [ ]:
fig, axes = plt.subplots(3, len(datasets), figsize=(10, 10))
for i, k in enumerate([2, 3, 4]):
    for j, data in enumerate(datasets):
        print(f"k = {k}, dataset #{j}")
        labels, iters, err = spectral_k_means(data, k, 10)
        axes[i,j].scatter(data[:, 0], data[:, 1], c=labels, cmap='rainbow')
        axes[i,j].set_title('$k=%d$, iter$=%d$' % (k, iters))

YOUR ANSWER HERE